In [26]:
#1.Pulls delivery restaurants details from Zomato
#2.Need to add the start url of a locality
#3.set the popular_locality

In [27]:
import requests
from bs4 import BeautifulSoup
import re
import pandas
from collections import OrderedDict

In [28]:
startUrl = "https://www.zomato.com/bangalore/delivery-in-indiranagar?ref_page=subzone"
popular_locality = "Indiranagar"

In [29]:
#Used headers/agent because the request was timed out and asking for an agent. 
#Using following code we can fake the agent.

def getSoup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    response = requests.get(url,headers=headers)
    
    content = response.content
    soup = BeautifulSoup(content,"html.parser")
    print(url)
    return soup

In [30]:
#max pages
def getMaxPages():
    soup = getSoup(startUrl)
    pages = []
    pagesTag = soup.find("div",attrs={"class":"col-l-4 mtop pagination-number"})
    pagesTag_temp = pagesTag.find_all("b",recursive=True)
    
    for p in pagesTag_temp:
        pages.append(p.get_text())
        
    return int(max(pages))


In [31]:

list_dataframes = []
currentpage = 1
maxPages = getMaxPages()

while currentpage <= maxPages:
    
    pageUrl = startUrl+"&page="+str(currentpage)
    soup = getSoup(pageUrl)
    list_rest = soup.find_all("div",attrs={"class": "col-s-12"})
    
    for rest in list_rest:
        dataframe = OrderedDict() #dict

        #restaurant link
        link = rest.find("a",attrs={"class": "result-title hover_feedback zred bold ln24 fontsize0"})
        dataframe["restaurant_link"] = link['href']
        
        soup = getSoup(link['href'])
        
        #locality
        dataframe["locality"] = popular_locality
        
        #restaurant title
        top_header_title = (soup.find("h1",attrs={"class":"ui res-name mb0 header nowrap"}))
        dataframe["restaurant_name"] = top_header_title.find("a").get_text().strip()

        #zomato gold
        goldTag = soup.find("div",attrs={"class":"red_res_tag"})
        if goldTag is not None:
            goldTag_temp = goldTag.find("img")
            if goldTag_temp is not None:
                dataframe["zomato_gold"] = "zomato gold" 

        #discounts
        discountTag = soup.find_all("div",attrs={"class": "fontsize flex flex-center-v"})
        for child in discountTag:
            discountTag_temp = child.find("div",attrs={"class":"pr5"})
            dataframe["discounts"] = discountTag_temp.getText()
            
        #photos
        photosTag = soup.find_all("div",attrs={"respageMenuContainer"})
        for photo in photosTag:
            children = photo.findChildren("a",attrs={"item respageMenu-item photosTab restaurant-tab-item-jumbo-track"})
            for child in children:
                reObj = re.compile('\d+')
                dataframe["photos_taken"] = reObj.findall(child.text.strip())[0]

        #rating
        top_header_rating = (soup.find("div",attrs={"class":"col-l-4 tac left"}))
        rating = top_header_rating.find("div",attrs={"class":"rating_hover_popup res-rating pos-relative clearfix mb5"})

        if rating is not None:
            dataframe["rating"] = rating.get_text().replace('\n',' ').strip()
        else:
            dataframe["rating"] = "NA"

        #votes
        top_header_votes = top_header_rating.find("span",attrs={"class":"mt2 mb0 rating-votes-div rrw-votes grey-text fontsize5 ta-right"})
        if top_header_votes is not None:
            dataframe["votes"] = top_header_votes.get_text()
        else:
            dataframe["votes"] = "NA"

        #cuisines
        info_left = (soup.find("div",attrs={"class":"col-l-1by3 pl0 pr20"}))
        cuisine_info = info_left.find("div",attrs={"class":"res-info-cuisines clearfix"})
        dataframe["cuisines"] = cuisine_info.get_text()

        #Approx cost
        approxCost_tag = info_left.find("div",attrs={"class":"res-info-detail"})
        if approxCost_tag is not None:
            approx_cost = approxCost_tag.find("span",attrs={"tabindex":"0"})
        if approx_cost is not None:
            reObj = re.compile('\d+')
            dataframe["approx_cost_for_2"] = reObj.findall(approx_cost.get_text())[0]
        else:
            dataframe["approx_cost_for_2"] = "NA"
        
        #Opening timings
        info_centre = (soup.find("div",attrs={"class":"col-l-1by3 pl20 pr20"}))
        if info_centre is not None:
            openingtimes = info_centre.find("div",attrs={"class":"res-info-timings"})
        if openingtimes is not None:
            timings = openingtimes.find("div",attrs={"class":"medium"})
            dataframe["opening_timings"] = re.findall(r'\s(\d*\:\d*\s?(?:AM|PM|am|pm|noon|midnight|Hours|hours)|\d*\s?(?:AM|PM|am|pm|noon|midnight|Hours|hours))',timings.get_text())
        
        #address
        addressTag = info_centre.find("div",attrs={"class":"mbot0"})
        address = addressTag.find("span")
        if addressTag and address is not None:
            dataframe["address"] = address.get_text()
        else:
            dataframe["address"] = "NA"

        #more info
        more_info = (soup.find_all("div",attrs={"class":"res-info-highlights"}))
        for child in more_info:
            infos = child.find_all("div",attrs={"class":"res-info-feature-text"})
            infostring = ''
            for info in infos:
                infostring += info.get_text()+', '
            dataframe["more_info"] = infostring

        #known for
        knownForTag = soup.find_all("div",attrs={"class":"mbot0 ptop0"})
        for child in knownForTag:
            known_for_list = child.find_all("div",attrs={"class":"res-info-known-for-text mr5"})
            for known_for in known_for_list:
                dataframe["known_for"] = known_for.get_text().strip()

        #loved here food
        mostlovedTags = soup.find_all("div",attrs={"class":"rv_highlights__wrapper mtop0"})
        for child in mostlovedTags:
            j = child.find_all("div",attrs={"class":"rv_highlights__section pr10"})
        for op in j:
            mostlovedcategory = op.find("div",attrs={"class":"grey-text"})

            loved_here = op.find("div",attrs={"class":"rv_highlights__score_bar mt5 mb5"})
            blockbg = loved_here.find_all("div",attrs={"class":"block bd-txt-bg"})

            mostLoved = op.find("div",attrs={"class":"fontsize13 ln18"})
            if mostLoved is not None:
                item = mostLoved.find_all("span")
                if item is not None:
                    temp = []
                    temp.append(5-len(blockbg))
                    for i in item:
                        temp.append(i.get_text().replace(',','').strip())
                    dataframe["most_liked_"+mostlovedcategory.get_text()] = temp 

        ##reviews
        review_link = link['href']+"/reviews"
        review_soup = getSoup(review_link)

        review_header = (review_soup.find_all("div",attrs={"class":"res-reviews-container res-reviews-area"}))
        review_list = []
        for i in review_header:
            rev_children = i.findChildren("div",attrs={"rev-text mbot0"})
            for child in rev_children:
                review_list.append(child.text.replace('Rated\xa0\n',' ').strip())

        dataframe["reviews"] = review_list
        
        list_dataframes.append(dataframe)

    currentpage += 1
else:
    currentpage = 1
        



https://www.zomato.com/bangalore/delivery-in-indiranagar?ref_page=subzone
https://www.zomato.com/bangalore/delivery-in-indiranagar?ref_page=subzone&page=1
https://www.zomato.com/bangalore/eat-fit-indiranagar-bangalore
https://www.zomato.com/bangalore/eat-fit-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/the-twisted-desi-indiranagar-bangalore
https://www.zomato.com/bangalore/the-twisted-desi-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/paradise-biryani-2-indiranagar-bangalore
https://www.zomato.com/bangalore/paradise-biryani-2-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/potful-2-domlur
https://www.zomato.com/bangalore/potful-2-domlur/reviews
https://www.zomato.com/bangalore/burger-king-indiranagar
https://www.zomato.com/bangalore/burger-king-indiranagar/reviews
https://www.zomato.com/bangalore/desi-chinese-kitchen-indiranagar-bangalore
https://www.zomato.com/bangalore/desi-chinese-kitchen-indiranagar-bangalore/reviews
https://www.zom

https://www.zomato.com/bangalore/cafe-grill-jeevan-bhima-nagar/reviews
https://www.zomato.com/bangalore/juicy-airport-road
https://www.zomato.com/bangalore/juicy-airport-road/reviews
https://www.zomato.com/bangalore/sagar-delicacy-ulsoor
https://www.zomato.com/bangalore/sagar-delicacy-ulsoor/reviews
https://www.zomato.com/bangalore/punjabi-nawabi-indiranagar-bangalore
https://www.zomato.com/bangalore/punjabi-nawabi-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/samosa-singh-thippasandra-bangalore
https://www.zomato.com/bangalore/samosa-singh-thippasandra-bangalore/reviews
https://www.zomato.com/bangalore/delivery-in-indiranagar?ref_page=subzone&page=5
https://www.zomato.com/bangalore/andhra-ruchulu-airport-road-bangalore
https://www.zomato.com/bangalore/andhra-ruchulu-airport-road-bangalore/reviews
https://www.zomato.com/manisindiranagar
https://www.zomato.com/manisindiranagar/reviews
https://www.zomato.com/bangalore/mojo-pizza-2x-toppings-indiranagar-bangalore
https://

https://www.zomato.com/bangalore/the-khichadi-company-1-indiranagar-bangalore
https://www.zomato.com/bangalore/the-khichadi-company-1-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/absolute-shawarma-thippasandra-bangalore
https://www.zomato.com/bangalore/absolute-shawarma-thippasandra-bangalore/reviews
https://www.zomato.com/bangalore/green-g-juice-bar-indiranagar
https://www.zomato.com/bangalore/green-g-juice-bar-indiranagar/reviews
https://www.zomato.com/bangalore/flaspo-fruit-flavoured-sponge-cakes-indiranagar-bangalore
https://www.zomato.com/bangalore/flaspo-fruit-flavoured-sponge-cakes-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/basil-spice-thippasandra-bangalore
https://www.zomato.com/bangalore/basil-spice-thippasandra-bangalore/reviews
https://www.zomato.com/bangalore/bangalore-dosa-co-jeevan-bhima-nagar-bangalore
https://www.zomato.com/bangalore/bangalore-dosa-co-jeevan-bhima-nagar-bangalore/reviews
https://www.zomato.com/bangalore/kurtosshhh-

https://www.zomato.com/bangalore/the-good-bowl-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/khichdi-experiment-indiranagar-bangalore
https://www.zomato.com/bangalore/khichdi-experiment-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/seafood-place-indiranagar
https://www.zomato.com/bangalore/seafood-place-indiranagar/reviews
https://www.zomato.com/bangalore/indian-spice-food-ulsoor-bangalore
https://www.zomato.com/bangalore/indian-spice-food-ulsoor-bangalore/reviews
https://www.zomato.com/bangalore/dal-bati-corner-thippasandra-bangalore
https://www.zomato.com/bangalore/dal-bati-corner-thippasandra-bangalore/reviews
https://www.zomato.com/bangalore/nandos-indiranagar
https://www.zomato.com/bangalore/nandos-indiranagar/reviews
https://www.zomato.com/bangalore/yauatcha-mg-road
https://www.zomato.com/bangalore/yauatcha-mg-road/reviews
https://www.zomato.com/bangalore/tea-brew-indiranagar
https://www.zomato.com/bangalore/tea-brew-indiranagar/reviews
https://w

https://www.zomato.com/bangalore/delivery-in-indiranagar?ref_page=subzone&page=16
https://www.zomato.com/bangalore/beijing-bites-indiranagar-bangalore
https://www.zomato.com/bangalore/beijing-bites-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/brahmins-thatte-idli-indiranagar-bangalore
https://www.zomato.com/bangalore/brahmins-thatte-idli-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/chai-resto-1-jeevan-bhima-nagar-bangalore
https://www.zomato.com/bangalore/chai-resto-1-jeevan-bhima-nagar-bangalore/reviews
https://www.zomato.com/bangalore/classic-lassi-shop-ulsoor
https://www.zomato.com/bangalore/classic-lassi-shop-ulsoor/reviews
https://www.zomato.com/bangalore/tayabbs-indiranagar
https://www.zomato.com/bangalore/tayabbs-indiranagar/reviews
https://www.zomato.com/bangalore/homely-kitchen-indiranagar-bangalore
https://www.zomato.com/bangalore/homely-kitchen-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/shawarma-inc-indiranagar-bangalor

...

https://www.zomato.com/bangalore/zawaa-foods-airport-road-bangalore
https://www.zomato.com/bangalore/zawaa-foods-airport-road-bangalore/reviews
https://www.zomato.com/bangalore/shanmukha-airport-road-bangalore
https://www.zomato.com/bangalore/shanmukha-airport-road-bangalore/reviews
https://www.zomato.com/bangalore/mac-fast-food-airport-road
https://www.zomato.com/bangalore/mac-fast-food-airport-road/reviews
https://www.zomato.com/bangalore/swaadpurna-jeevan-bhima-nagar
https://www.zomato.com/bangalore/swaadpurna-jeevan-bhima-nagar/reviews
https://www.zomato.com/bangalore/delivery-in-indiranagar?ref_page=subzone&page=57
https://www.zomato.com/bangalore/cafe-velvette-domlur
https://www.zomato.com/bangalore/cafe-velvette-domlur/reviews
https://www.zomato.com/bangalore/aysha-restaurant-jeevan-bhima-nagar
https://www.zomato.com/bangalore/aysha-restaurant-jeevan-bhima-nagar/reviews
https://www.zomato.com/bangalore/ambur-hot-dum-biryani-airport-road
https://www.zomato.com/bangalore/ambur-hot

https://www.zomato.com/bangalore/fudgy-love-designer-cakes-domlur-bangalore
https://www.zomato.com/bangalore/fudgy-love-designer-cakes-domlur-bangalore/reviews
https://www.zomato.com/bangalore/ambur-hot-dum-biriyani-jeevan-bhima-nagar
https://www.zomato.com/bangalore/ambur-hot-dum-biriyani-jeevan-bhima-nagar/reviews
https://www.zomato.com/bangalore/bhikaram-chandmal-bhujiawala-jeevan-bhima-nagar
https://www.zomato.com/bangalore/bhikaram-chandmal-bhujiawala-jeevan-bhima-nagar/reviews
https://www.zomato.com/bangalore/abbus-biryani-thippasandra-bangalore
https://www.zomato.com/bangalore/abbus-biryani-thippasandra-bangalore/reviews
https://www.zomato.com/bangalore/bombay-kulfis-jeevan-bhima-nagar-bangalore
https://www.zomato.com/bangalore/bombay-kulfis-jeevan-bhima-nagar-bangalore/reviews
https://www.zomato.com/bangalore/cravebelly-veg-non-veg-airport-road-bangalore
https://www.zomato.com/bangalore/cravebelly-veg-non-veg-airport-road-bangalore/reviews
https://www.zomato.com/bangalore/banga

https://www.zomato.com/bangalore/blue-tokai-coffee-roasters-indiranagar-bangalore
https://www.zomato.com/bangalore/blue-tokai-coffee-roasters-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/the-corner-hyatt-centric-mg-road-ulsoor-bangalore
https://www.zomato.com/bangalore/the-corner-hyatt-centric-mg-road-ulsoor-bangalore/reviews
https://www.zomato.com/bangalore/vietnamese-kitchen-indiranagar
https://www.zomato.com/bangalore/vietnamese-kitchen-indiranagar/reviews
https://www.zomato.com/bangalore/hunger-addict-airport-road-bangalore
https://www.zomato.com/bangalore/hunger-addict-airport-road-bangalore/reviews
https://www.zomato.com/bangalore/obiryani-1-ejipura-bangalore
https://www.zomato.com/bangalore/obiryani-1-ejipura-bangalore/reviews
https://www.zomato.com/bangalore/meet-delights-commercial-street-bangalore
https://www.zomato.com/bangalore/meet-delights-commercial-street-bangalore/reviews
https://www.zomato.com/bangalore/the-souk-ulsoor
https://www.zomato.com/bangalor

https://www.zomato.com/bangalore/wowffles-indiranagar-bangalore
https://www.zomato.com/bangalore/wowffles-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/the-real-taste-1-indiranagar-bangalore
https://www.zomato.com/bangalore/the-real-taste-1-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/try-once-kitchen-1-indiranagar-bangalore
https://www.zomato.com/bangalore/try-once-kitchen-1-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/juicebox-indiranagar-bangalore
https://www.zomato.com/bangalore/juicebox-indiranagar-bangalore/reviews
https://www.zomato.com/bangalore/sri-ganesh-bangarpet-chats-and-sweets-btm-bangalore
https://www.zomato.com/bangalore/sri-ganesh-bangarpet-chats-and-sweets-btm-bangalore/reviews
https://www.zomato.com/bangalore/delivery-in-indiranagar?ref_page=subzone&page=68
https://www.zomato.com/bangalore/fab-food-24-7-btm-bangalore
https://www.zomato.com/bangalore/fab-food-24-7-btm-bangalore/reviews
https://www.zomato.com/bangalo

AttributeError: 'NoneType' object has no attribute 'find'

In [32]:
df = pandas.DataFrame(list_dataframes,columns=list_dataframes[0].keys())
df.to_csv("zomato_csvs/zomato_"+popular_locality+".csv",index=False)